#1. Library imports

In [1]:
#!pip install transformers
#!pip install datasets

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import tqdm
import nltk
import torch
import numpy as np
import pandas as pd
from torch import nn
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import confusion_matrix, classification_report
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
import transformers

In [ ]:
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import tqdm
from sklearn.metrics import confusion_matrix, classification_report

# 2. Data Imports

## Vanilla

In [ ]:
def load_data(split_name='train', columns=['text', 'stars'], folder='gdrive/MyDrive/COMP_4332/Project1/data'):
        df = pd.read_csv(f'{folder}/{split_name}.csv')
        return df

In [ ]:
train_df = load_data('train', columns=['text', 'stars'])
valid_df = load_data('valid', columns=['text', 'stars'])
# the test set labels (the 'stars' column) are not available! So the following code will instead return all columns
test_df = load_data('test', columns=['text', 'stars'])

In [ ]:
train_df = train_df[["text","stars"]]
valid_df = valid_df[["text","stars"]]

## Pre-process texts

In [ ]:
stopword_list = set(stopwords.words('english'))
ps = PorterStemmer()

def lower(s):
    """
    :param s: a string.
    return a string with lower characters
    Note that we allow the input to be nested string of a list.
    e.g.
    Input: 'Text mining is to identify useful information.'
    Output: 'text mining is to identify useful information.'
    """
    if isinstance(s, list):
        return [lower(t) for t in s]
    if isinstance(s, str):
        return s.lower()
    else:
        raise NotImplementedError("unknown datatype")


def tokenize(text):
    """
    :param text: a doc with multiple sentences, type: str
    return a word list, type: list
    e.g.
    Input: 'Text mining is to identify useful information.'
    Output: ['Text', 'mining', 'is', 'to', 'identify', 'useful', 'information', '.']
    """
    return nltk.word_tokenize(text)


def stem(tokens):
    """
    :param tokens: a list of tokens, type: list
    return a list of stemmed words, type: list
    e.g.
    Input: ['Text', 'mining', 'is', 'to', 'identify', 'useful', 'information', '.']
    Output: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.']
    """
    ### equivalent code
    # results = list()
    # for token in tokens:
    #     results.append(ps.stem(token))
    # return results

    return [ps.stem(token) for token in tokens]

def n_gram(tokens, n=1):
    """
    :param tokens: a list of tokens, type: list
    :param n: the corresponding n-gram, type: int
    return a list of n-gram tokens, type: list
    e.g.
    Input: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.'], 2
    Output: ['text mine', 'mine is', 'is to', 'to identifi', 'identifi use', 'use inform', 'inform .']
    """
    if n == 1:
        return tokens
    else:
        results = list()
        for i in range(len(tokens)-n+1):
            # tokens[i:i+n] will return a sublist from i th to i+n th (i+n th is not included)
            results.append(" ".join(tokens[i:i+n]))
        return results

def filter_stopwords(tokens):
    """
    :param tokens: a list of tokens, type: list
    return a list of filtered tokens, type: list
    e.g.
    Input: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.']
    Output: ['text', 'mine', 'identifi', 'use', 'inform', '.']
    """
    ### equivalent code
    # results = list()
    # for token in tokens:
    #     if token not in stopwords and not token.isnumeric():
    #         results.append(token)
    # return results

    return [token for token in tokens if token not in stopword_list and not token.isnumeric()]

import numpy as np

def get_onehot_vector(feats, feats_dict):
    """
    :param data: a list of features, type: list
    :param feats_dict: a dict from features to indices, type: dict
    return a feature vector,
    """
    # initialize the vector as all zeros
    vector = np.zeros(len(feats_dict), dtype=np.float)
    for f in feats:
        # get the feature index, return -1 if the feature is not existed
        f_idx = feats_dict.get(f, -1)
        if f_idx != -1:
            # set the corresponding element as 1
            vector[f_idx] = 1
    return vector

In [ ]:
train_df['tokens'] = train_df['text'].map(tokenize).map(filter_stopwords).map(lower)
valid_df['tokens'] = valid_df['text'].map(tokenize).map(filter_stopwords).map(lower)
test_df['tokens'] = test_df['text'].map(tokenize).map(filter_stopwords).map(lower)

## Dataset Class

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df):

    
    def __getitem__(self, idx):

    def __len__(self):


In [ ]:
train_Dataloader = DataLoader(MyDataset(x_train, y_train), batch_size=32,shuffle=True)
valid_Dataloader = DataLoader(MyDataset(valid_x, valid_y), batch_size=32)

# 3. Models

In [ ]:
train_df = load_data('train', columns=['text', 'stars'])
valid_df = load_data('valid', columns=['text', 'stars'])
# the test set labels (the 'stars' column) are not available! So the following code will instead return all columns
test_df = load_data('test', columns=['text', 'stars'])

In [ ]:
train_df.columns

In [ ]:
x_train = train_df['text']
y_train = train_df['stars']
  
x_valid = valid_df['text']
y_valid = valid_df['stars']

x_test = test_df['text']

## Naive Solutions

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=tokenize)#analyzer=lambda x: x

In [ ]:
tfidf_concat = tfidf.fit_transform(x_train['text'])

In [ ]:
x_train['text_vector']
x_train.head()

### Naive-Bayes

In [ ]:
tfidf = TfidfVectorizer(tokenizer=tokenize)#analyzer=lambda x: x
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
steps = [('tfidf', tfidf),('mnb', mnb)]
pipe = Pipeline(steps)
print(pipe)

In [ ]:
pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_valid)
print(classification_report(y_valid, y_pred))
print("\n\n")
print(confusion_matrix(y_valid, y_pred))
print('accuracy', np.mean(y_valid == y_pred))

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
steps = [('tfidf', tfidf),('dt', dt)]
pipe = Pipeline(steps)
print(pipe)

In [ ]:
pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_valid)
print(classification_report(y_valid, y_pred))
print("\n\n")
print(confusion_matrix(y_valid, y_pred))
print('accuracy', np.mean(y_valid == y_pred))

## Default MLP Classifier

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
steps = [('tfidf', tfidf),('dt', mlp)]
pipe = Pipeline(steps)
print(pipe)

In [ ]:
pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_valid)
print(classification_report(y_valid, y_pred))
print("\n\n")
print(confusion_matrix(y_valid, y_pred))
print('accuracy', np.mean(y_valid == y_pred))

## Gradient Boosting

In [ ]:
vocab = TfidfVectorizer(tokenizer=tokenize)#analyzer=lambda x: x
x_train_gb = vocab.fit_transform(x_train)
X_valid_gb = vocab.fit_transform(x_valid)
x_test_gb = vocab.fit_transform(x_test)

In [ ]:
"""
from sklearn.ensemble import GradientBoostingClassifier
gbe = GradientBoostingClassifier(random_state=0)
parameters = {
     'learning_rate': [0.05, 0.1, 0.5],
    'max_features': [0.25,0.5, 1],
    'max_depth': [2,3, 4, 5]}
gridsearch=GridSearchCV(gbe,parameters,cv=100,scoring='roc_auc')
gridsearch.fit(x_train_gb,y_train)
print(gridsearch.best_params_)
print(gridsearch.best_score_)
"""

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
#grad_boost = GradientBoostingClassifier(**gridsearch.best_params)
grad_boost = GradientBoostingClassifier(learning_rate=0.05,max_depth=5,random_state=1002)
steps = [('tfidf', tfidf),('dt', grad_boost)]
pipe = Pipeline(steps)
print(pipe)

In [ ]:
pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_valid)
print(classification_report(y_valid, y_pred))
print("\n\n")
print(confusion_matrix(y_valid, y_pred))
print('accuracy', np.mean(y_valid == y_pred))

## XGBoost

In [ ]:
import xgboost
from xgboost import XGBClassifier
xg_boost = XGBClassifier()
steps = [('tfidf', tfidf),('xgboost', xg_boost)]
pipe = Pipeline(steps)
print(pipe)

In [ ]:
pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_valid)
print(classification_report(y_valid, y_pred))
print("\n\n")
print(confusion_matrix(y_valid, y_pred))
print('accuracy', np.mean(y_valid == y_pred))

# 4. Advanced Model (with finetuning)

In [ ]:
train_df.head()
valid_df.head()
#test_df.head()

,text,stars
0,We came in today during closing & they still a...,5
1,"Tiny, but casual location for breakfast/brunch...",4
2,We keep going to the same plaza to eat pizza b...,4
3,Tim Hortons is the epitome of Canadian mediocr...,1
4,Workers here are very friendly and they know a...,5


### Testing with Dataset format

In [ ]:
"""
from transformers import pipeline
classifier = pipeline("sentiment-analysis")
#e1 = classifier("Hello my name is Sun Bin, and I am very excited to start this code!")
"""

In [ ]:
from datasets import load_dataset
raw = load_dataset('yelp_review_full')

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/13c31a618ba62568ec8572a222a283dfc29a6517776a3ac5945fb508877dde43. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
small_train_dataset = raw["train"].shuffle(seed=1002).select(range(2000))
small_eval_dataset = raw["test"].shuffle(seed=42).select(range(1000))

In [ ]:
#small_train_dataset = train_df_tokenized
#small_eval_dataset = valid_df_tokenized

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve/main/voc

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from datasets import load_metric
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

finetune_bert = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)
train_args = TrainingArguments(output_dir="test_trainer",
                               evaluation_strategy = 'epoch')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

metric = load_metric("accuracy")

trainer = Trainer(
    model=finetune_bert,
    args=train_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-cased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/092cc582560fc3833e556b3f833695c26343cb54b7e88cd02d40821462a74999.1f48cab6c959fc6c360d22bea39d06959e90f5b002e77e836d2da45464875cda
creating metadata file for /root/.cache/huggingface/transformers/092cc582560fc3833e556b3f833695c26343cb54b7e88cd02d40821462a74999.1f48cab6c959fc6c360d22bea39d06959e90f5b002e77e836d2da45464875cda
loading weights file https://huggingface.co/bert-base-cased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/092cc582560fc3833e556b3f833695c26343cb54b7e88cd02d40821462a74999.1f48cab6c959fc6c360d22bea39d06959e90f5b002e77e836d2da45464875cda
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.trans

RuntimeError: ignored

### Custom Dataset

In [ ]:
train_dict = []
for i, j in zip(list(train_df["stars"]),list(train_df["text"])):
    train_dict.append({"label": i, "text": j})
valid_dict = []
for i, j in zip(list(valid_df["stars"]),list(valid_df["text"])):
    valid_dict.append({"label": i, "text": j})

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_texts = list(train_df["text"])
valid_texts = list(valid_df["text"])
test_texts = list(test_df["text"])

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(valid_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
import torch

class TestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_label = train_df["stars"]
valid_label = valid_df["stars"]

train_dataset = TestDataset(train_encodings, train_label)
val_dataset = TestDataset(val_encodings, valid_label)
#test_dataset = TestDataset(test_encodings, test_labels)

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

finetune_distilbert = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased') 

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=finetune_distilbert,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

RuntimeError: ignored

In [ ]:
import torch

class MyDataset:
    def __init__(self, df):
        self.label = list(df['stars'])
        self.text = list(df['text'])
    def __getitem__(self,idx):
      return np.asarray(self.label[idx]), np.asarray(self.text[idx])
      return item
    def __len__(self):
      return len(self.label)

"""
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
"""

new_train = MyDataset(train_df)

In [ ]:
#new_train.text
#new_train.label

### Failed Attempts (Running out of GPU)

In [ ]:
#train_df_tokenized = tokenizer(list(train_df["text"]), padding="max_length", truncation=True)

In [ ]:
#valid_df_tokenized = tokenizer(list(valid_df["text"]), padding="max_length", truncation=True)

In [ ]:
batch_size = 8
train_dataloader = DataLoader(MyDataset(train_df), batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(MyDataset(valid_df), batch_size=batch_size)

In [ ]:
from datasets import load_metric
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

finetune_bert = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)
train_args = TrainingArguments(output_dir="test_trainer",
                               evaluation_strategy = 'epoch')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

metric = load_metric("accuracy")

trainer = Trainer(
    model=finetune_bert,
    args=train_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)


Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
finetune_bert

# Experiment (RoBERTa)

In [8]:
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import tqdm
from sklearn.metrics import confusion_matrix, classification_report

In [9]:
checkpoint = "cardiffnlp/twitter-roberta-base-emotion"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.model_max_len=512

Downloading:   0%|          | 0.00/768 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [14]:
#tokenize dataset
def tokenize(text):
  return tokenizer(text, truncation=True,max_length=512,padding='max_length')

def tokenize_df(df):
  tokens = df['text'].map(tokenize)
  df['input_ids'] = [x['input_ids'] for x in tokens]
  df['attention_mask'] = [x['attention_mask'] for x in tokens]
  return df

train_df = tokenize_df(train_df)
valid_df = tokenize_df(valid_df)
test_df = tokenize_df(test_df)

In [16]:
class MyDataset(Dataset):
    def __init__(self, df):
        assert len(df['input_ids']) == len(df['stars'])
        self.input_ids = df['input_ids']
        self.attention_mask = df['attention_mask']
        self.label = df['stars']-1
    
    def __getitem__(self, idx):
        return np.asarray(self.input_ids[idx]), np.asarray(self.attention_mask[idx]), self.label[idx]

    def __len__(self):
        return len(self.label)

In [24]:
train_df[]

,text,stars,input_ids,attention_mask
0,I've been here a handful of times now and I've...,5,"[0, 100, 348, 57, 259, 10, 6095, 9, 498, 122, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,The service was terrible. The food was just ok...,1,"[0, 133, 544, 21, 6587, 4, 20, 689, 21, 95, 15...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,Alil pricey for the location but completly get...,4,"[0, 7083, 718, 26428, 13, 5, 2259, 53, 14156, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Don't get your car washed here. Paid 11 and my...,1,"[0, 6766, 75, 120, 110, 512, 15158, 259, 4, 37...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,Cute but tight. Not expensive and creative. I ...,5,"[0, 347, 4467, 53, 3229, 4, 1491, 3214, 8, 390...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
17995,DHL is not my friend. The instructions to clea...,1,"[0, 495, 8064, 16, 45, 127, 1441, 4, 20, 9223,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
17996,It's a nicely laid out shop with lots of jewel...,3,"[0, 243, 18, 10, 15481, 4976, 66, 2792, 19, 37...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
17997,Called the salon to advise the owner that my p...,1,"[0, 347, 9315, 5, 21425, 7, 12922, 5, 1945, 14...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
17998,If you like authentic mexican seafood - than t...,4,"[0, 1106, 47, 101, 12757, 162, 1178, 12657, 17...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [22]:
x = MyDataset(train_df)
x.__getitem__(2)

(array([    0,  7083,   718, 26428,    13,     5,  2259,    53, 14156,
           90,   352,   120,     5, 14225,    13,   110, 13182,  4045,
        22391,    15,   477,   727,   207,     2,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
      

In [17]:
batch_size = 4

train_dataloader = DataLoader(MyDataset(train_df), batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(MyDataset(valid_df), batch_size=batch_size)